In [18]:
!pip install -qU crewai[tools,agentops]==0.95.0

In [19]:
!pip install -qU tavily-python scrapegraph-py

In [20]:
!pip install -qU litellm

In [21]:
from crewai import Agent, Task, Crew, Process, LLM
from crewai.tools import tool

import agentops
import os
from google.colab import userdata

from pydantic import BaseModel, Field
from typing import List
from tavily import TavilyClient

In [22]:
GROQ_API_KEY=userdata.get('GROQ_API_KEY')
os.environ["GROQ_API_KEY"] =userdata.get('GROQ_API_KEY')
os.environ["AGENTOPS_API_KEY"] = userdata.get('AGENT_OPS_API_KEY')


In [23]:
from litellm import completion
import os

os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')
response = completion(
    model="groq/llama-3.1-70b-versatile",
    messages=[
       {"role": "user", "content": "hello from litellm"}
   ],
)
print(response)

ModelResponse(id='chatcmpl-47134420-9836-427c-8d9e-cddbad8cd8e4', created=1736967591, model='groq/llama-3.1-70b-versatile', object='chat.completion', system_fingerprint='fp_4e32347616', choices=[Choices(finish_reason='stop', index=0, message=Message(content="Hello from the other side. It's nice to meet you, Litellm. How can I assist you today or would you like to chat?", role='assistant', tool_calls=None, function_call=None))], usage=Usage(completion_tokens=31, prompt_tokens=40, total_tokens=71, completion_tokens_details=None, prompt_tokens_details=None, queue_time=0.7827918460000001, prompt_time=0.007730818, completion_time=0.112727273, total_time=0.120458091), x_groq={'id': 'req_01jhnnc591e0sra68yps70jvs1'})


In [24]:
basic_llm = LLM(
    model="groq/llama-3.1-70b-versatile",
    temperature=0
)
search_client = TavilyClient(api_key=userdata.get('TVLY_API_KEY'))

In [25]:

agentops.init(
    api_key=userdata.get('AGENT_OPS_API_KEY'),
    skip_auto_end_session=True
)

🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=453839fb-31c1-457a-8c55-8a4bccfb08e2


In [38]:
#end_state = "success" 
#agentops.end_session(end_state)


🖇 AgentOps: Invalid end_state. Please use one of the EndState enums


In [26]:
output_dir = "./ai-agent-output"
os.makedirs(output_dir, exist_ok=True)

In [31]:
no_keywords = 15

list_make_cars = [
    "Make",
    "Abarth",
    "Alfa Romeo",
    "Aston Martin",
    "Audi",
    "Bentley",
    "BMW",
    "Cadillac",
    "Changan",
    "Chevrolet",
    "Dodge",
    "Ferrari",
    "Fiat",
    "Ford",
    "Hyundai",
    "Infiniti",
    "Jaguar",
    "Jeep",
    "KIA",
    "Lamborghini",
    "Land Rover",
    "Lexus",
    "Lincoln",
    "Maserati",
    "Maybach",
    "Mazda",
    "McLaren",
    "Mercedes-Benz",
    "MG",
    "Mini",
    "Mitsubishi",
    "Nissan",
    "Porsche",
    "Range Rover",
    "Renault",
    "Rolls Royce",
    "Subaru",
    "Suzuki",
    "Tesla",
    "Toyota",
    "Volkswagen",
    "Volvo"
]

type_list = [
    "Type",
    "Hatchback",
    "Sedan",
    "Coupe",
    "Convertible",
    "Crossover",
    "SUV",
    "Van",
    "Pickup"
]
websites_list =["https://www.arabam.com/" , "https://www.cars.com/" , "https://www.arabalar.com.tr/"]
product_name = "Car"


In [29]:
class SuggestedSearchQueries(BaseModel):
    queries: List[str] = Field(
        ...,
        title="Suggested search queries",
        description="A list of search queries for the search engine.",
        min_items=1,
        max_items=10
    )

search_queries_recommendation_agent = Agent(
    role="Search Queries Recommendation Agent",
    goal="\n".join([
        "Generate a list of targeted search queries based on user preferences, platform needs, and contextual data." ,
        "To provide a list of suggested search queries to be passed to the search engine.",
        "The queries must be varied and looking for specific items."
    ]),
    backstory=(
        "This agent helps users of Murshid by suggesting search queries tailored to finding the right car listings."
    ),
    llm=basic_llm,
    verbose=True,
)
search_queries_recommendation_task = Task(
    description="\n".join([
        "The user is searching for a car with the following requirements:",
        "- Price under {max_price}.",
        "- Manufacturer: {make_car}.",
        "The user wants to buy {product_name} at the best prices (value-for-price strategy).",
        "Target websites for purchase: {websites_list}.",
        "All available products on the internet should be included for comparison at a later stage.",
        "The stores must sell the product in {country_name}.",
        "Generate a maximum of {no_keywords} queries.",
        "Search queries must be written in the {language} language.",
        "Queries must specify the car type: {car_type}. Avoid using general keywords.",
        "The search query must lead to an e-commerce product page, not blogs or listing pages."
    ]),
    expected_output="A JSON object containing a list of suggested search queries.",
    output_json=SuggestedSearchQueries,
    output_file=os.path.join(output_dir, "step_1_suggested_search_queries.json"),
    agent=search_queries_recommendation_agent
)

In [32]:
murshid_crew = Crew(
    agents=[
        search_queries_recommendation_agent
    ],
    tasks=[
        search_queries_recommendation_task
    ],
    process=Process.sequential,
)

In [34]:
crew_results = murshid_crew.kickoff(
    inputs={
        "max_price":"10000$" ,
        "make_car": "KIA",
        "product_name":product_name,
        "websites_list":websites_list,
        "country_name": "Egypt",
        "no_keywords": 10,
        "language": "English" ,
        "car_type": "Sedan"

    }
)

# Agent: Search Queries Recommendation Agent
## Task: The user is searching for a car with the following requirements:
- Price under 10000$.
- Manufacturer: KIA.
The user wants to buy Car at the best prices (value-for-price strategy).
Target websites for purchase: ['https://www.arabam.com/', 'https://www.cars.com/', 'https://www.arabalar.com.tr/'].
All available products on the internet should be included for comparison at a later stage.
The stores must sell the product in Egypt.
Generate a maximum of 10 queries.
Search queries must be written in the English language.
Queries must specify the car type: Sedan. Avoid using general keywords.
The search query must lead to an e-commerce product page, not blogs or listing pages.


# Agent: Search Queries Recommendation Agent
## Final Answer: 
{
  "queries": [
    "KIA Sedan for sale in Egypt under 10000",
    "Buy KIA Sedan in Egypt price under 10000",
    "KIA Sedan models in Egypt for sale at best price",
    "Cheap KIA Sedan in Egypt for 

In [ ]:
class SignleSearchResult(BaseModel):
    title: str
    url: str = Field(..., title="the page url")
    content: str
    score: float
    search_query: str

class AllSearchResults(BaseModel):
    results: List[SignleSearchResult]

@tool
def search_engine_tool(query: str):
    """Useful for search-based queries. Use this to find current information about any query related pages using a search engine"""
    return search_client.search(query)

search_engine_agent = Agent(
    role="Search Engine Agent",
    goal="To search for products based on the suggested search query",
    backstory="This agent helps users of Murshid by suggesting search queries tailored to finding the right car listings.",
    llm=basic_llm,
    verbose=True,
    tools=[search_engine_tool]
)

search_engine_task = Task(
    description="\n".join([
        "The task is to search for products based on the suggested search queries.",
        "You have to collect results from multiple search queries.",
        "Ignore any susbicious links or not an ecommerce single product website link.",
        "Ignore any search results with confidence score less than ({score_th}) .",
        "The search results will be used to compare prices of products from different websites.",
    ]),
    expected_output="A JSON object containing the search results.",
    output_json=AllSearchResults,
    output_file=os.path.join(output_dir, "step_2_search_results.json"),
    agent=search_engine_agent
)

In [ ]:
earch_engine_agent = Agent(
    role="Search Engine Agent",
    goal="To search for products based on the suggested search query",
    backstory="The agent is designed to help in looking for products by searching for products based on the suggested search queries.",
    llm=basic_llm,
    verbose=True,
    tools=[search_engine_tool]
)